# JSON Conversions

This notebook demonstrates how to [convert](https://nexus-forge.readthedocs.io/en/latest/interaction.html#converting) a [Resource](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource) to JSON and vice-versa. 

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
import json

In [4]:
from kgforge.core import Resource

In [5]:
def pp(x): print(json.dumps(x, indent=4))

## Resource to JSON

In [6]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [7]:
jane = Resource(type="Person", name="Jane Doe", address=address)

In [8]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [9]:
persons = [jane, john]

In [10]:
forge.register(persons)

<count> 2
<action> _register_many
<succeeded> True


In [11]:
association = Resource(type="Association", agent=persons)

In [12]:
forge.register(association)

<action> _register_one
<succeeded> True


In [13]:
pp(forge.as_json(association, store_metadata=True))

{
    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/101a874a-0e59-404f-aa89-4e9fc9119520",
    "type": "Association",
    "agent": [
        {
            "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/caccdb9d-a374-4ab4-85b1-01efb1d8d5e9",
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva"
            },
            "name": "Jane Doe"
        },
        {
            "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/e3817898-d82c-442c-a53e-6367fde62beb",
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ],
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_createdAt": "2022-04-12T22:22:23.988Z",
    "_createdBy": "https://bbp.epfl.ch/nexus/v1/realms/bbp/users/sy",
    "_deprecated": false,
    "_incoming": "https://bb

## JSON to Resource

In [14]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [15]:
jane = Resource(type="Person", name="Jane Doe", address=address, award=["Nobel"])

In [16]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [17]:
persons = [jane, john]

In [18]:
association = Resource(type="Association", agent=[jane, john])

In [19]:
data = {
    "type": "Association",
    "agent": [
        {
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva",
            },
            "email": "(missing)",
            "name": "Jane Doe",
            "award":["Nobel"]
        },
        {
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ]
}

In [20]:
resource = forge.from_json(data, na="(missing)")

In [21]:
resource == association

True